In [1]:
%load_ext autoreload
%autoreload 2

import molpy as mp
import numpy as np
from pathlib import Path

data_path = Path("data/case2")

In [2]:
struct_hd = mp.Monomer.from_frame(mp.io.read_pdb(data_path / "hexamethylenediamine.pdb"), "hd")
struct_aa = mp.Monomer.from_frame(mp.io.read_pdb(data_path / "adipic_acid.pdb"), "aa")

# typifier = mp.typifier.AmberToolsTypifier("gaff", "bcc")
# typifier.typify(struct_hd, workdir=data_path / "hd")
# typifier.typify(struct_aa, workdir=data_path / "aa")

In [3]:
# rxn1_end1 = struct_hd.get_substruct(indices=[1, 2, 3, 4, 5, 7, 8, 15, 19, 20])
# rxn1_end2 = struct_aa.get_substruct(indices=[1, 3, 6, 9, 4, 5, 7, 8, 10, 11, 13, 14])

In [ ]:
site_a = struct_hd.def_link_site(struct_hd["atoms"][0], )
site_b = struct_aa.def_link_site(struct_aa["atoms"][0], )
reacter = mp.reacter.LammpsReacter(mp.typifier.AmberToolsTypifier("gaff", "bcc"))
struct_stp1 = reacter.react(
    "stp1",
    struct_hd,
    struct_aa,
    site_a,
    site_b,
    struct_hd["atoms"][8],
    struct_aa["atoms"][8],
    workdir=data_path
)

<Struct: 9 atoms>

In [ ]:
frame = struct_rxn1.to_frame(bond_keys=['i', 'j'])
for bond in frame["bonds"].iterrows():
    print(bond['i'], bond['j'])

NameError: name 'struct_rxn1' is not defined

Usage: antechamber -i     input file name
                   -fi    input file format
                   -o     output file name
                   -fo    output file format
                   -c     charge method
                   -cf    charge file name
                   -nc    net molecular charge (int)
                   -a     additional file name
                   -fa    additional file format
                   -ao    additional file operation
                          crd   : only read in coordinate
                          crg   : only read in charge
                          radius: only read in radius
                          name  : only read in atom name
                          type  : only read in atom type
                          bond  : only read in bond type 
                   -m     multiplicity (2S+1), default is 1
                   -rn    residue name, overrides input file, default is MOL
                   -rf    residue toplogy file name in prep input f

In [ ]:
struct_rxn1.del_bond()
typifier.typify(struct_rxn1, workdir=data_path / "rxn1")
mp.io.write_lammps_molecule(data_path / "rxn1_stp2_before.mol", struct_rxn1)

<Struct: 44 atoms>

In [ ]:
struct_rxn1.def_atoms()  # delete water
mp.io.write_lammps_molecule(data_path / "rxn1_stp2_after.mol", struct_rxn1)

In [ ]:
# packing and run simulation